#### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
    
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import time
import re


In [2]:
def hh_pars_page(hh_xml, data):
    ''' 
    Парсинг HH-lxml страницы и добавление данных в pd.data
    '''

    list_block = hh_xml.find_all('div',{"class":"vacancy-serp-item"})


    for div in list_block:
        vacancy = div.find('a', {'class': 'bloko-link HH-LinkModifier'}).text
        #print(vacancy)
        vacancy_link = div.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        #print(vacancy_link)
        employer = div.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).text
        #print(employer)

        compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        if not compensation:
            compensation1 = np.NaN
            compensation2 = np.NaN
        else:
            compensation = compensation.text.replace('.', '').replace(' ','').replace('руб','')
           # print(compensation)
            if '-' in compensation:
                compensations = compensation.split('-')

                compensation1 = compensations[0] #.replace(r'\D', '').replace(r'\xa', '')
                compensation1 = re.sub(r'\D', '',compensation1)
                compensation2 = compensations[1] #.replace(r'\D', '').replace(r'\xa', '')
                compensation2 = re.sub(r'\D', '',compensation2)
            elif 'от' in compensation:
                compensation1 = compensation.replace('от','').replace(r'\xa', '')
                compensation1 = re.sub(r'\D', '',compensation1)
                compensation2 = np.NaN
            elif 'до' in compensation:
                compensation1 = np.NaN
                compensation2 = compensation.replace('до','').replace(' ', '')
                compensation2 = re.sub(r'\D', '',compensation2)

        #print(compensation1, compensation2)

        data = data.append({'vacancy':[vacancy], 'min_salary':[compensation1], 'max_salary':[compensation2], 'vacancy_link':[vacancy_link], 'link':['https://hh.ru'], 
                            'employer':[employer]}, ignore_index=True)

        #compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    return(data)
    

In [3]:
def sj_pars_page(sj_xml, data):
    ''' 
    Парсинг SJ-lxml страницы и добавление данных в pd.data
    '''

    list_block = sj_xml.find_all('div',{"class":"_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"})
    #pprint(list_block)
    for div in list_block:
        vacancy = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
       # print(vacancy)
        vacancy_link = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).findParent()['href']
        vacancy_link = 'https://www.superjob.ru' + vacancy_link                               
        #print(vacancy_link)
        employer = div.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI'}).text
        #print(employer)     

        compensation = div.find('span', {'class': '_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).text
        #print(compensation)
        
        if '—' in compensation:

            compensation1 = compensation.split('—')[0]
            compensation2 = compensation.split('—')[1]
            compensation1 = re.sub(r'\D', '',compensation1)
            compensation2 = re.sub(r'\D', '',compensation2)

        elif 'от' in compensation:
            compensation1 = compensation[3:]
            compensation1 = re.sub(r'\D', '',compensation1)
            #print("compensation1=", compensation1)
            compensation2 = np.NaN
        elif 'договорённости' in compensation:
            compensation1 = np.NaN
            compensation2 = np.NaN
        elif 'до' in compensation:
            compensation1 = np.NaN
            compensation2 = compensation[3:]
       
        data = data.append({'vacancy':[vacancy], 'min_salary':[compensation1], 'max_salary':[compensation2], 'vacancy_link':[vacancy_link], 'link':['https://www.superjob.ru'], 
                            'employer':[employer]}, ignore_index=True)

        #compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    return(data)
    

In [4]:
# Общие параметры поиска: текст вакансии и глубина
search_text = "методолог"
pages = 4
data = pd.DataFrame()

In [6]:
#  Обработка HH

main_link = ('https://hh.ru')
method = "/search/vacancy?area=1&st=searchVacancy&text="
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

url = main_link+method+search_text
print(main_link+method+search_text)

session = requests.Session()

for i in range(pages):
    html = session.get(url, headers=headers).text
    
    hh_xml = bs(html,'lxml')
    data = hh_pars_page(hh_xml, data)
    url = main_link + hh_xml.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    
    time.sleep(1)
    
    

https://hh.ru/search/vacancy?area=1&st=searchVacancy&text=методолог


In [9]:
#  Обработка SJ

main_link = ('https://www.superjob.ru')
method = "/vacancy/search/?keywords="
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

url = main_link+method+search_text
print(main_link+method+search_text)

session = requests.Session()


for i in range(pages):
    html = session.get(url, headers=headers).text
    
    sj_xml = bs(html,'lxml')
    data = sj_pars_page(sj_xml, data)
    url = main_link + sj_xml.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'})['href']
    
    time.sleep(1)


https://www.superjob.ru/vacancy/search/?keywords=методолог


In [10]:
data

,employer,link,max_salary,min_salary,vacancy,vacancy_link
0,[ Всероссийский Банк Развития Регионов (ВБРР)],[https://hh.ru],[nan],[nan],[Методолог кадрового администрирования],[https://hh.ru/vacancy/34240183?query=%D0%BC%D...
1,"[ Ренессанс cтрахование, Группа]",[https://hh.ru],[nan],[nan],[Ведущий специалист/Риск-менеджер в Управление...,[https://hh.ru/vacancy/34195483?query=%D0%BC%D...
2,[ Банк Русский Стандарт],[https://hh.ru],[nan],[nan],[Методолог],[https://hh.ru/vacancy/34246226?query=%D0%BC%D...
3,[ Группа компаний ПИК],[https://hh.ru],[nan],[nan],[Методолог],[https://hh.ru/vacancy/33913089?query=%D0%BC%D...
4,[ООО СТРАТЕГИЯ],[https://hh.ru],[150000],[nan],[Методолог/Финансовый Бизнес аналитик],[https://hh.ru/vacancy/33223198?query=%D0%BC%D...
5,[ РОСГОССТРАХ Банк],[https://hh.ru],[nan],[nan],[Методолог Отдела методологии и отчетности в ц...,[https://hh.ru/vacancy/33786376?query=%D0%BC%D...
6,[ Факторинговая компания НФК],[https://hh.ru],[120000],[nan],[Ведущий бизнес-аналитик (методолог)],[https://hh.ru/vacancy/31830250?query=%D0%BC%D...
7,[ Отраслевой центр разработок и внедрения инфо...,[https://hh.ru],[120000],[120000],[Методолог-консультант],[https://hh.ru/vacancy/33772466?query=%D0%BC%D...
8,[ Юниты],[https://hh.ru],[nan],[nan],[Методолог],[https://hh.ru/vacancy/33767200?query=%D0%BC%D...
9,[ ТРИНИТИ],[https://hh.ru],[100000],[100000],[Методолог],[https://hh.ru/vacancy/33754084?query=%D0%BC%D...
